In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from torchvision.models import resnet18
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from scipy.optimize import linear_sum_assignment
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget

from data import get_bloodmnist_loader, get_simsiam_loader
from simsiam import SimSiam

!nvidia-smi

/h/lkcai/anaconda3/envs/video_comp_py38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Thu Nov 28 15:00:36 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:86:00.0 Off |                    0 |
| N/A   47C    P8    16W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
train_loader = get_bloodmnist_loader(split='train', batch_size=64, ratio=0.1)
val_loader = get_bloodmnist_loader(split='val', batch_size=100)

unsupervised_loader = get_simsiam_loader()

Using downloaded and verified file: /h/lkcai/.medmnist/bloodmnist_128.npz
Using downloaded and verified file: /h/lkcai/.medmnist/bloodmnist_128.npz
Using downloaded and verified file: /h/lkcai/.medmnist/bloodmnist_128.npz


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SimSiam()
model = model.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=0.05, momentum=0.9, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()

def pretrain_simsiam(model, dataloader, optimizer, device, epochs=50):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for (images1, images2), _ in tqdm(dataloader):
            images1 = images1.to(device)
            images2 = images2.to(device)
            
            outputs = model(images1, images2)
            loss = outputs['loss']

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {total_loss / len(dataloader):.4f}")

pretrain_simsiam(model, unsupervised_loader, optimizer, device, epochs=100)


100%|██████████| 187/187 [09:01<00:00,  2.89s/it]


Epoch [1/10], Loss: -0.6593


 58%|█████▊    | 109/187 [05:23<03:51,  2.97s/it]


KeyboardInterrupt: 

In [ ]:
linear_prob_model = nn.Sequential(
    model.backbone,
    nn.Linear(1024, 8)
)
finetune_model = finetune_model.to(device)

finetune_optimizer = torch.optim.Adam(finetune_model.parameters(), lr=0.01)
finetune_criterion = nn.CrossEntropyLoss()

def finetune_model_on_labeled_data(model, train_loader, val_loader, optimizer, criterion, device, epochs=20):
    model.train()
    for epoch in range(epochs):
        train_loss, correct, total = 0, 0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        val_acc = evaluate_model(model, val_loader, device)
        print(f"Epoch [{epoch + 1}/{epochs}], Train Loss: {train_loss / len(train_loader):.4f}, "
              f"Train Acc: {100. * correct / total:.2f}%, Val Acc: {val_acc:.2f}%")

def evaluate_model(model, dataloader, device):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    return 100. * correct / total

finetune_model_on_labeled_data(
    finetune_model, labeled_loader, val_loader, finetune_optimizer, finetune_criterion, device, epochs=20)
